In [1]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score
import sklearn.metrics
import scipy as sp
from scipy import sparse
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV

# 1. Feature engineering

In [2]:
def data2sparse(data, num_heroes, names):
    
#    r_new = []
#    for i in xrange(num_heroes):
#        r_new += ['r'+str(i+1)+'hero','r'+str(i+1)+'level', 'r'+str(i+1)+'xp', 'r'+str(i+1)+'gold', 'r'+str(i+1)+'lh',
#             'r'+str(i+1)+'kills','r'+str(i+1)+'deaths','r'+str(i+1)+'items', 'd'+str(i+1)+'hero', 'd'+str(i+1)+'level', 
#             'd'+str(i+1)+'xp', 'd'+str(i+1)+'gold', 'd'+str(i+1)+'lh','d'+str(i+1)+'kills','d'+str(i+1)+'deaths',
#             'd'+str(i+1)+'items']
    
    
    data=data.fillna(0)
    r = data.index
    for j in ['r','d']:  
        for i in xrange(5):
            index=j+str(i+1)+'_hero'
            r = np.vstack([r,data[index]*16-16])
    r = np.transpose(r[1:])    

    row = []
    col = []
    val = []
    for i in xrange(len(data.index)):
#        print len(data)
        for k in xrange(5):
            for j in xrange(8):
                row.insert(j+8*k+i*1000,i)
                col.insert(j+8*k+i*1000,r[i,k]+j)
                val.insert(j+8*k+i*1000,data[names[3+j+8*k]][data.index[i]])
#                print i, k, j, names[3+j+8*k]
                row.insert(j+8*(k+5)+i*1000,i)
                col.insert(j+8*(k+5)+i*1000,r[i,(k+5)]+j)
                val.insert(j+8*(k+5)+i*1000,-data[names[3+j+8*(k+5)]][data.index[i]])
    
#    return val, row, col
    
    nele=max(row)+1
    nbus=max(col)+1
    data_sparse = sparse.coo_matrix((val, (row,col)), shape=(nele, nbus), dtype=float)
    
    del num_heroes, val, row, col
    return data_sparse
    

In [3]:
def role_add(data, heroes):
    type_vector = list(heroes.columns)[3:]
    heroes['sum']=1.*heroes[type_vector].sum(axis=1)
    for i in xrange(len(type_vector)):
        heroes[type_vector[i]]=np.divide(heroes[type_vector[i]],heroes['sum'])
    del heroes['sum']
    new = []
    r_old = []
    d_old = []
    type_vector = list(heroes.columns)[3:]
    attr = ['_hero', '_level', '_xp', '_gold', '_lh', '_kills', '_deaths', '_items']
    for k in type_vector:
        for i in attr:
            new += [k+i]
    for k in range(0,5):
        for i in attr:
            r_old+=['r'+str(k+1)+i] 
    for k in range(0,5):
        for i in attr:
            d_old+=['d'+str(k+1)+i] 
    for i in list(new):
        data[i]=0
    type_vector = list(heroes.columns)[3:12]
    for j in xrange(5):
        R = pd.merge(data[['match_id',r_old[8*j]]],heroes,left_on=r_old[8*j],right_on='id', how='left')
        D = pd.merge(data[['match_id',d_old[8*j]]],heroes,left_on=d_old[8*j],right_on='id', how='left')
        R.index=data.index
        D.index=data.index
        for i in xrange(9):
            for k in xrange(8):
                x=type_vector[i]
                if k == 0:
                    data[new[8*i+k]]+=R[x]
                    data[new[8*i+k]]-=D[x]
                else:
                    data[new[8*i+k]]+=R[x]*data[r_old[8*j+k]]
                    data[new[8*i+k]]-=D[x]*data[d_old[8*j+k]] 
    return data

# 2. Classification

In [126]:
features = pd.read_csv("features.csv")
features_check = pd.read_csv("features_test.csv")
name_vector = list(features_check.columns)[2:]
features = features.fillna(0)
features_check = features_check.fillna(0)
data = features
sub = features_check
y = data['radiant_win']
del(features)
del(features_check)
heroes = pd.read_csv('dictionaries/heroes2.csv', ';')
num_heroes = len(data['r1_hero'].unique()) 
names = list(data.columns.values)

In [127]:
data = role_add(data, heroes)

### Sparse Logistics

In [128]:
train, test, y_train, y_test = train_test_split(
...     data, y, test_size=0.3, random_state=1)

In [129]:
del data

In [132]:
#from sklearn.preprocessing import normalize

In [133]:
#from sklearn.naive_bayes import GaussianNB
#from sklearn.naive_bayes import MultinomialNB

In [134]:
#w = np.ones(len(train))/len(train)

In [135]:
#model1 = LogisticRegressionCV(Cs=10, penalty='l1', tol=0.001, cv=4, solver="liblinear")

In [130]:
X_train1 = data2sparse(train[train.columns[:103]], num_heroes, names)

In [145]:
X_train= np.hstack([X_train1.toarray(), train[train.columns[109:]]])

In [146]:
del X_train1

In [147]:
X_train= np.hstack([X_train, train[train.columns[84:103]]])

In [148]:
X_train.shape

(68061, 1875)

In [149]:
X_test1 = data2sparse(test[train.columns[:103]], num_heroes, names)

In [150]:
X_test= np.hstack([X_test1.toarray(), test[train.columns[109:]], test[train.columns[84:103]]])

In [151]:
del X_test1

In [152]:
#from sklearn.preprocessing import normalize

In [153]:
#X_test = normalize(X_test)

In [154]:
#X_train = normalize(X_train[:20000])

In [155]:
model1 = LogisticRegression(C=0.03, penalty='l1', tol=0.01, solver="liblinear")

In [156]:
model1.fit(X_train,y_train)

LogisticRegression(C=0.03, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.01,
          verbose=0, warm_start=False)

In [157]:
pred1 = model1.predict_proba(X_test)[:,1]
print roc_auc_score(y_test, pred1)

0.757145097779


### Roles classifier

In [66]:
X_train = train[train.columns[109:]]
X_test = test[train.columns[109:]]

In [44]:
model2 = LogisticRegression(C=0.017, penalty='l1', tol=0.01, random_state=142)
model2.fit(X_train,y_train)

LogisticRegression(C=0.017, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l1', random_state=142, solver='liblinear', tol=0.01,
          verbose=0)

In [45]:
pred1 = model2.predict_proba(X_test)[:,1]
print roc_auc_score(y_test, pred1)

0.725042769455


In [67]:
e2 = 1-model2.score(X_test, y_test)
a2 = 0.5*np.log((1-e2)/e2)

### Parameters classifier

In [50]:
X_train = train[train.columns[84:103]]
X_test = test[train.columns[84:103]]

In [51]:
model3 = LogisticRegression(C=0.017, penalty='l1', tol=0.01, random_state=142)

In [52]:
model3.fit(X_train,y_train)

LogisticRegression(C=0.017, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l1', random_state=142, solver='liblinear', tol=0.01,
          verbose=0)

In [53]:
pred1 = model3.predict_proba(X_test)[:,1]
print roc_auc_score(y_test, pred1)

0.625170895186


In [54]:
w3 = model3.score(X_test, y_test)

In [55]:
print w3

0.590729884466


### Row players data classifier

In [20]:
X_train = train[train.columns[:83]]
X_test = test[train.columns[:83]]

In [21]:
model4 = LogisticRegression(C=0.017, penalty='l1', tol=0.01, random_state=142)

In [22]:
model4.fit(X_train,y_train)

LogisticRegression(C=0.017, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l1', random_state=142, solver='liblinear', tol=0.01,
          verbose=0)

In [23]:
pred1 = model4.predict_proba(X_test)[:,1]
print roc_auc_score(y_test, pred1)

0.713816036967


# 7. Gradient boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score

In [ ]:
model2 = GradientBoostingClassifier(n_estimators=300, verbose=True, random_state=241, learning_rate=0.3)
model2.fit(X_train,y_train)

In [ ]:
pred1 = model1.predict_proba(X_test.toarray())[:, 1]
pred2 = model2.predict_proba(X_test.toarray())[:, 1]
print roc_auc_score(y_test, pred1)
print roc_auc_score(y_test, pred2)